In [1]:
#import dependencies
import pandas as pd

In [3]:
# read in data from CSV and create a dataframe

filepath = "2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv"

file = pd.read_csv(filepath)

df = pd.DataFrame(file)

df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [4]:
# create a copy dataframe to work with

sales_df = df.copy()

### This assignment states that there seems to be something wrong with a previously calculated average order value so the first step is to sort the data by order amount and look for any orders that seem unusual.

In [29]:
# sort the data by order_amount from greatest to least and inspect the results

sorted_df = sales_df.sort_values("order_amount",axis=0,ascending=False)

# uncomment the following line to see the data
# sorted_df.head(50)

In [30]:
# look at all orders by id 607

df_607 = sales_df.loc[sales_df['user_id'] == 607]

#uncomment the following line to see the data
# df_607

In [31]:
# check to see if there are seemingly normal transactions at store 42 where all the 607 transactions occur
df_42 = sales_df.loc[sales_df['shop_id'] == 42]

#uncomment the following line to see the data
# df_42

### Right away it becomes clear that there are some transactions that would throw off the calculation of average order value, and they fall into two types. The first type is a series of transactions by a single user id, 607, all done by credit card at store 42. Each transaction by id 607 is for 2,000 items at a cost of 704,000, which averages to 352/item. Other smaller transactions occurred at store 42 with similar prices per item so the issue does not seem to be the store. Ideally, I would need to find out more information about user id 607 in order to handle it properly but without being able to do that treating 607 as an outlier and excluding all transactions by user id 607 from calculations is the best option. 

In [32]:
# drop all transactions by user id 607 from the dataframe

step_one = sorted_df[sorted_df.user_id != 607]

#uncomment the following line to see the data
# step_one.head(50)

In [33]:
# check for normal transactions at store 78

df_78 = sales_df.loc[sales_df['shop_id'] == 78]

#uncomment the following line to see the data
# df_78

### The second type of problematic transaction is one where the number of items purchased and the order amount are significantly out of proportion. These transactions all occured at shop 78 and no transactions that appear reasonable happened at this shop. It would be helpful to be able to look into shop 78 but since this is not possible, the best option is to exclude all data from this shop in order to avoid skewing the results of AOV calculations. (Best guess here: either purchase data was entered incorrectly and 25725 for one item should be 257.25 for one item or this store was closing and these are the result of selling off fixtures and other assets.)

In [34]:
# exclude data from store 78
cleaned_df = step_one[step_one.shop_id != 78]

#uncomment the following line to see the data
# cleaned_df.head(50)


### Now that the data is cleaned, we can calculate an AOV that more accurately reflects normal business in the stores. 

In [28]:
# find the total number of transactions, the total sales, and then calculate AOV

transactions = cleaned_df["order_id"].nunique()

total_sales = cleaned_df["order_amount"].sum()

aov = round(total_sales/transactions,2)

aov

302.58

### When calculated with cleaned data, the average order value is 302.58, a much more reasonable amount for transactions in shoe stores than the original calculation given in the assignment. The anomalies of user id 607 and shop 78 should still be accounted for in order to have a complete picture.